# <a href="https://thetahat.ru/courses/bm-2024-aut">Cтатистика ФБМФ </a>

## Семинар 11

In [9]:
# Bot check

# HW_ID: st_sem11
# Бот проверит этот ID и предупредит, если случайно сдать что-то не то.

# Status: final
# Перед отправкой в финальном решении удали "not" в строчке выше.
# Так бот проверит, что ты отправляешь финальную версию, а не промежуточную.
# Никакие значения в этой ячейке не влияют на факт сдачи работы.

In [1]:
import pandas as pd
import scipy.stats as sps
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
# зафиксируем сид для воспроизводимости генерации
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

## Множественная проверка гипотез

С помощью статистических методов можно проверить человека на наличие экстрасенсорных способностей: предложим ему угадать последовательность, состоящую из двух цветов, длины 100.

Сформулируем задачу на статистическом языке:

$X_1...X_{100}$ &mdash; выборка из распределения $Bern(p)$

$p=0.5$ отвечает случайному угадыванию.

Проверьте гипотезу: $\mathsf{H}_0 \colon p=0.5$ vs $\mathsf{H}_1 \colon p \neq 0.5$. Используйте критерий Вальда.

В качестве асимптотически нормальной оценки можно использовать $\widehat{p} = \overline{X}$ с асимптотической дисперсией $\sigma^2(p) = p (1 - p)$.

Выпишем состоятельную оценку дисперсии и статистику критерия Вальда:

$\widehat{\sigma} = \sqrt{\overline{X} (1 - \overline{X})}$, $W = \sqrt{n} \frac{\overline{X} - 0.5}{\sqrt{\overline{X} (1 - \overline{X})}}$

Оценим реальный уровень значимости для этого критерия при размере выборки равном 100. К чему он должен быть близок?

**Ответ:** к 0.05

Для скорости вычислений используйте количество выборок равное $10^3$.

In [2]:
sample_size = 100
sample_count = 1000

theta = 0.5

In [3]:
def wald_test(sample, theta, estimation_theta, estimation_sigma, alpha=0.05, alternative='two_sided'):
    """
    param sample: реализация выборки
    param theta: истинное значение параметра
    param estimation_theta: оценка параметра
    param estimation_sigma: оценка асимптотической дисперсии оценки estimation_sigma
    param alpha: уровень значимости критерия
    param alternative: вид альтернативной гипотезы, может принимать одно из значений 'two_sided', 'less', 'greater'

    return statistic
    return p_value
    """

    alpha = 0.05
    z = sps.norm.ppf(1 - alpha/2)
    n = len(sample)
    statistic = np.sqrt(n) * (estimation_theta - theta) / estimation_sigma

    if alternative == 'two_sided':
        p_value = sps.norm.sf(np.abs(statistic)) + sps.norm.cdf(-np.abs(statistic))
        conf_int = round(estimation_theta - z*estimation_sigma/np.sqrt(n), 4), round(estimation_theta + z*estimation_sigma/np.sqrt(n), 4)


    elif alternative == 'less':
        p_value = sps.norm.cdf(statistic)
        conf_int = (-np.inf, round(estimation_theta + z*estimation_sigma/np.sqrt(n), 4))


    elif alternative == 'greater':
        p_value = sps.norm.sf(statistic)
        conf_int = (round(estimation_theta + z*estimation_sigma/np.sqrt(n), 4), np.inf)

    else:
        raise ValueError('alternative name is wrong')

    return statistic, p_value, conf_int

Оценим реальный уровень значимости

In [4]:
sample = sps.bernoulli(p = 0.5).rvs(size = (sample_count, sample_size))

estimation_theta = sample.mean(axis = 1)
estimation_sigma = np.sqrt(estimation_theta * (1 - estimation_theta))


counter = 0

for i in range(sample_count):
    _, p_value, conf_int = wald_test(sample[i], theta, estimation_theta[i], estimation_sigma[i])
    is_rejected = p_value < 0.05
    if is_rejected:
        counter += 1

counter / sample_count



0.059

Знаачение действительно близко к 0.05

Теперь представим, что мы хотим проверить большое количество людей на экстрасенсорные способности с помощью данного критерия.

Проведите аналогичный эксперимент: сгенерируйте $10^3$ выборок размера $100$ для $100$ людей. Посчитайте, сколько раз из 1000 в вашем наборе из 100 выборок хотя бы для одной гипотеза будет отвергнута.

In [5]:
sample_people = 100
sample_all = sps.bernoulli(p = 0.5).rvs(size = (100, 1000, 100))

counter = 0
for sample in sample_all:
    estimation_theta = sample.mean(axis = 1)
    estimation_sigma = np.sqrt(estimation_theta * (1 - estimation_theta))


    for i in range(100):
        _, p_value, conf_int = wald_test(sample[i], theta, estimation_theta[i], estimation_sigma[i])
        is_rejected = p_value < 0.05
        if is_rejected:
            counter += 1
            break

In [6]:
counter / 1000

0.1

**Вывод:** 10% - это большой процент отвергнутый гипотез при $\alpha = 0.05$ (критерий не применим). Вероятно, так получилось, потому что мы берем уровень значимости для одного эксперимента, а идет параллельно 100 экспериментов, и у нас накапливается ошибка первого рода.

На лекции вы прошли метод, позволяющий не накапливать ошибку 1 рода. В этом методе необходимо использовать уровень значимости, зависящий от количества проверяемых одновременно гипотез.

**Чему равен этот уровень значимости, если одновременно проверяются n гипотез?**

**Ответ:** $\frac{\alpha}{n}$, где $\alpha$ - уровень значимости для одной гипотезы.

Проведите предыдущий эксперимент с использованием корректной процедуры. Поскольку в реализованной выше функции $\alpha$ зафиксировано, используйте критерий отвержения гипотезы с помощью p-value.

В данном задании **ЗАПРЕЩЕНО** пользоваться `statsmodels.stats.multitest.multipletests`

In [7]:
sample_all = sps.bernoulli(p = 0.5).rvs(size = (100, 1000, 100))

counter = 0
for sample in sample_all:
    estimation_theta = sample.mean(axis = 1)
    estimation_sigma = np.sqrt(estimation_theta * (1 - estimation_theta))

    for i in range(100):
        _, p_value, conf_int = wald_test(sample[i], theta, estimation_theta[i], estimation_sigma[i])
        is_rejected = p_value < (0.05 / 100)
        if is_rejected:
            counter += 1
            break

In [8]:
counter / 1000

0.009

**Вывод:** Видно, что процент отвергнутых гипотез уменьшился. Критерий стал применим, когда мы стали делить уровень значимости на количество экспериментов.